In [ ]:
import pandas as pd
l = pd.read_csv('../data/lucknow.csv')
l

In [ ]:
l['date'] = pd.to_datetime(l['date'])
l.set_index('date', inplace=True)
l

In [ ]:
from statsmodels.tsa.seasonal import seasonal_decompose
l_AQI = l['AQI']
result = seasonal_decompose(l_AQI, model='multiplicative')
result.plot();

In [ ]:
!pip install pmdarima
import pmdarima as pm

In [ ]:
from statsmodels.tsa.statespace.sarimax import SARIMAX
from pmdarima import auto_arima;

In [ ]:
l_AQI = l['AQI']
auto_arima(y=l_AQI,start_p=0,start_P=0,start_q=0,start_Q=0,seasonal=True, m=12)

In [ ]:
train = l_AQI[:45] #from 2015-2018
test = l_AQI[46:65]# july 2018-june 2019

In [ ]:
test

In [ ]:
model=SARIMAX(train,order=(1,0,0),seasonal_order=(1,0,1,12),)
results=model.fit()
results.summary()

In [ ]:
predictions = results.predict(start=45, end=67, typ='levels').rename('Predictions')

In [ ]:
predictions.plot(legend=True)
test.plot(legend=True,title="lucknow Prediction data");

In [ ]:
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score, explained_variance_score, max_error,mean_poisson_deviance,mean_gamma_deviance
import math
import numpy as np
RMSE=np.sqrt(mean_squared_error(predictions,test))
print('Root Mean Squared Error: ', RMSE)
print('Mean AQI:',test.mean())
forecast_errors = [test[i]-predictions[i] for i in range(len(test))]
bias = sum(forecast_errors) * 1.0/len(test)
print('Bias: %f' % bias)
mse = mean_squared_error(test, predictions)
print('MSE: '+str(mse))
mae = mean_absolute_error(test, predictions)
print('MAE: '+str(mae))
rmse = math.sqrt(mean_squared_error(test, predictions))
print('RMSE: '+str(rmse))
mape = np.mean(np.abs(predictions - test)/np.abs(test))
print('MAPE: '+str(mape))
r2score=r2_score(test, predictions)
print('r2score: '+str(r2score))
explainedVariance_score=explained_variance_score(test, predictions)
print('explainedVariance_score: '+str(explainedVariance_score))
me=max_error(test, predictions)
print('me: '+str(me))
mpd=mean_poisson_deviance(test, predictions)
print('mpd: '+str(mpd))
mgd=mean_gamma_deviance(test, predictions)
print('mgd: '+str(mgd))

In [ ]:
# Forming the model:
final_model = SARIMAX(train,order=(1,0,0),seasonal_order=(1,0,1,12))
results = final_model.fit()
#Obtaining predicted values:
predictions = results.predict(start=64, end=77, typ='levels').rename('Predictions')
#Plotting predicted values against the true values:
predictions.plot(legend=True)
l_AQI.plot(legend=True,figsize=(12,8),grid=True,title="Delhi AQI");

In [15]:
import pickle
pickle.dump(results, open('lucknow.pkl', 'wb'))